In [52]:
import pandas as pd
import numpy as np
import scipy.stats as stats

df_main = pd.read_csv('payback.csv')

df_main.drop('user_id', axis=1, inplace = True)
df_main.sort_values(by = 'offer_id', ascending=True)
df_main.dropna(inplace = True)

# Сформируем общий список партнеров
df_offer_ids_list = df_1['offer_id'].unique()

print(df_offer_ids_list)

df_main

[  78   79   97   98  121  126  138  139  140  148  149  150  152  153
  157  234  235  236  238  239  243  247  249  251  284  300  473  474
  478  482  490  492  493  505  510  511  550  551  552  553  608  609
  610  611  612  613  621  622  624  625  626  627  629  630  666  810
  834  856  868  869  871  875  913  914  916  973  974  975 1056 1057
 1082 1087 1088 1104 1109 1154 1155 1156 1162 1233 1237 1272 1301 1303
 1342 1376 1380 1387 1388 1467 1592 1600 1605 1611 1623 1624 1625 1626
 1627 1632 1703 1704 1714 1906 1912 1913 1927 1947 2045 2080 2081 2089
 2153 2162 2178 2229 2249 2251 2252 2254 2256 2277 2281 2313 2346 2347
 2363 2403 2467 2470 2471 2472 2473 2482 2495 2496 2503 2510 2525 2529
 2532 2548 2633 2635 2646 2682 2685 2696 2697 2698 2699 2700 2720 2721
 2764 2779 2800 2806 2826 2830 2833 2838 2842 2845 2846 2852 2877 2891
 2892 2898 2915 2916 2918 2929 2933 2934 2935 2951 2953 2955 2958 2961
 2962 2963 2969 2970 2971 2977 2978 2985 2990 2992 2995 2997 2998 3021
 3027 

,offer_id,payback_day_avg
0,78,0.076
1,138,0.000
2,139,0.000
3,149,0.000
4,234,0.469
...,...,...
7721,6242,0.696
7722,6242,0.612
7723,6242,0.237
7724,6242,1.544


In [53]:
# Здесь основная цель - посчитать статистику по сервисам, а именно среднее и медианное значение 
# среднего срока окупаемости  для каждого сервиса
# Но как и в любом распределении, у нас могут быть выбросы. Уберем выбросы из статистики методом межквартильного
# анализа, предположив, что выброс - это значение превышающее размах между 1 (25%) и 3(75%) квартилем в 1,5 раза

# Создадим три массива - основной, массив для очищенных средних значений, массив для очищенных медианных значений
# и массив для id сервиса

massive = []
mean_clean = []
median_clean =[]
service_id=[]

service_list = {}

# Для каждого оффера наполним и очистим массивы из нашего основного датафрейма

for item in df_offer_ids_list:
    df_i = df_main[df_main['offer_id'] == item]
    massive = df_i['payback_day_avg'].to_frame().reset_index()    
    massive.drop(['index'], axis=1, inplace=True)
    q1 = massive.quantile(q=0.25)
    q3 = massive.quantile(q=0.75)
    qr = massive.apply(stats.iqr)
    massive_clean = massive[~((massive < (q1-1.5*qr)) | (massive > (q3+1.5*qr))).any(axis=1)]
    
    service_list[item] = massive_clean['payback_day_avg'].to_numpy()
    
    mean_clean.append(massive.mean()['payback_day_avg'])    
    median_clean.append(massive.median()['payback_day_avg'])
    service_id.append(item)
df_final = pd.DataFrame({'offer_id': service_id, "mean": mean_clean, "median": median_clean})
df_final.set_index('offer_id', inplace=True)
print(df_final)
service_list

              mean  median
offer_id                  
78        0.114835  0.0000
79        0.049263  0.0000
97        0.096017  0.0000
98        0.146200  0.0000
121       0.325500  0.3255
...            ...     ...
4535      0.800000  0.8000
4542      1.225000  1.2250
6242      1.310615  0.9050
9339      2.192960  1.7630
9340      2.970824  2.3420

[602 rows x 2 columns]


{78: array([0.076, 0.192, 0.   , 0.404, 0.   , 0.   , 0.038, 0.   , 0.   ,
        0.397, 0.019, 0.   , 0.   , 0.29 , 0.   , 0.25 , 0.226, 0.   ,
        0.   , 0.396, 0.194, 0.   , 0.   , 0.   , 0.259, 0.   , 0.019,
        0.   , 0.   , 0.   , 0.397, 0.   , 0.   , 0.   , 0.01 , 0.   ,
        0.   , 0.   , 0.   , 0.405, 0.   , 0.198, 0.398, 0.378, 0.   ,
        0.238, 0.   , 0.   , 0.   , 0.   , 0.   , 0.317, 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.378, 0.072, 0.396, 0.   , 0.   ,
        0.214, 0.   , 0.   , 0.155, 0.   , 0.397, 0.   , 0.   , 0.   ,
        0.079, 0.397, 0.399, 0.397, 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.386, 0.397, 0.   , 0.   , 0.   , 0.429,
        0.395, 0.327, 0.397, 0.   , 0.429, 0.   , 0.   , 0.   , 0.   ,
        0.031, 0.106, 0.429, 0.429, 0.571, 0.224]),
 79: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 97: array([0.

In [76]:
# Определим значение t-критерия для заданной точности и степеней свободы (n - 1). 
# Вычислим доверительные интервалы для прогнозирования среднего срока окупаемости. Но только в правом хвосте распределения.
# То есть 75й процентиль и 90й процентиль

service_id_massive = []

no_data_ids = []

up_bound75_massive = []

up_bound90_massive = []
mean_massive = []

for key, item in service_list.items():
    

    n = len(service_list[key])
    
    if n-1>0:
        mean = np.mean(service_list[key])
        mean_massive.append(mean)
        st_dev = np.std(service_list[key], ddof = 1)
        stError = st_dev / np.sqrt(n)

        t75 = stats.t.ppf(0.75, n-1)
        t90 = stats.t.ppf(0.90, n-1)


        # Вычисление доверительных интервалов

        up_bound75 = round(mean + t75 * stError,3)

        up_bound90 = round(mean + t90 * stError,3)
        
        up_bound75_massive.append(up_bound75)

        up_bound90_massive.append(up_bound90)

        service_id_massive.append(key)
        
    else:
        no_data_ids.append(key)



df_intervals_final = pd.DataFrame({'offer_id': service_id_massive,
                                   "mean": mean_massive,                            
                                   "up_bound75": up_bound75_massive,
                                   "up_bound90": up_bound90_massive})
print(f"Список сервисов, для которых у нас нет достаточного количества данных для статистически достоверного прогноза" 
      f"среднего срока окупаемости подписок партнера на сервисе {no_data_ids}")
print('')
print('Статистически достоверный прогноз среднего срока окупаемости подписок на сервисе с точностями 75% и 90%') 
df_intervals_final

Список сервисов, для которых у нас нет достаточного количества данных для статистически достоверного прогнозасреднего срока окупаемости подписок партнера на сервисе [152, 153, 157, 473, 621, 629, 834, 856, 875, 973, 1104, 1162, 1233, 1237, 1272, 1301, 1342, 1380, 1600, 1714, 2045, 2178, 2229, 2256, 2277, 2313, 2346, 2347, 2467, 2471, 2472, 2473, 2482, 2495, 2496, 2529, 2532, 2682, 2685, 2697, 2830, 2846, 2892, 2915, 2933, 2934, 2951, 2958, 2963, 2969, 2970, 2971, 2978, 2985, 2992, 2995, 2997, 2998, 3088, 3089, 3112, 3115, 3121, 3122, 3144, 3160, 3194, 3209, 3210, 3295, 3296, 3356, 3378, 3390, 3447, 3448, 3463, 3473, 3542, 3550, 3751, 3753, 3811, 3817, 3857, 3885, 3886, 3919, 3925, 3926, 3946, 3949, 3966, 3977, 3980, 3982, 3983, 3997, 4001, 4019, 4022, 4027, 4035, 4039, 4051, 4108, 4116, 4117, 4124, 4125, 4158, 4173, 4175, 4177, 4195, 4203, 4248, 4249, 4263, 4264, 4303, 4334, 4336, 4337, 4350, 4356, 4357, 4361, 4376, 4385, 4387, 4393, 4397, 4413, 4433, 4434, 4436, 4438, 4442, 4444, 4452

,offer_id,mean,up_bound75,up_bound90
0,78,0.109956,0.121,0.130
1,79,0.000000,0.000,0.000
2,97,0.000000,0.000,0.000
3,98,0.000467,0.001,0.001
4,121,0.325500,0.651,1.327
...,...,...,...,...
434,4535,0.800000,0.800,0.800
435,4542,1.225000,1.225,1.225
436,6242,0.890518,0.933,0.972
437,9339,1.688000,1.800,1.903


In [79]:
df_campaigns = pd.read_csv(r'Users_campaigns.csv')


,user_id,offer_id,campaign_id,total_profit,total_expense,payback_day
0,1,78,65607,105840,8000,0.076
1,1,138,65608,14641,0,0.000
2,1,139,65608,41895,0,0.000
3,1,149,65608,128037,0,0.000
4,1,234,65607,8954,4200,0.469
...,...,...,...,...,...,...
15950,42103,6242,291248,39825,19200,0.482
15951,42103,6242,293084,675,2400,3.556
15952,42103,6242,293769,270675,160800,0.594
15953,49444,6242,311440,60075,38400,0.639


In [96]:
df_campaigns_fin = df_campaigns.copy()
df_campaigns_fin.drop(['total_profit','total_expense'], axis=1, inplace = True)
df_campaigns_fin = df_campaigns_fin.merge(df_intervals_final, on='offer_id')
df_campaigns_fin['passed_75'] = np.where(df_campaigns_fin['payback_day'] <= df_campaigns_fin['up_bound75'], True, False)
df_campaigns_fin['passed_90'] = np.where(df_campaigns_fin['payback_day'] <= df_campaigns_fin['up_bound90'], True, False)
df_campaigns_fin_false75 = df_campaigns_fin[df_campaigns_fin['passed_75'] == False]
df_campaigns_fin_false90 = df_campaigns_fin[df_campaigns_fin['passed_90'] == False]
df_campaigns_fin_true = df_campaigns_fin[(df_campaigns_fin['passed_75'] == True) & (df_campaigns_fin['passed_90'] == True)]
df_campaigns_fin

,user_id,offer_id,campaign_id,payback_day,mean,up_bound75,up_bound90,passed_75,passed_90
0,1,78,65607,0.076,0.109956,0.121,0.13,True,True
1,8,78,2859,0.118,0.109956,0.121,0.13,True,True
2,8,78,3621,0.840,0.109956,0.121,0.13,False,False
3,8,78,7028,0.000,0.109956,0.121,0.13,True,True
4,8,78,7196,0.000,0.109956,0.121,0.13,True,True
...,...,...,...,...,...,...,...,...,...
15641,22984,4261,154770,0.800,0.800000,0.800,0.80,True,True
15642,25307,4261,152929,0.800,0.800000,0.800,0.80,True,True
15643,23379,3759,147465,0.800,0.800000,0.800,0.80,True,True
15644,23690,3759,145421,0.800,0.800000,0.800,0.80,True,True


In [94]:
# -------------ИТОГИ------------------------
# 1) Список партнеров с их рекламными кампаниями, которые прошли точку невозврата с точки зрения окупаемости с точностью 75%
df_campaigns_fin_false75


,user_id,offer_id,campaign_id,payback_day,mean,up_bound75,up_bound90,passed_75,passed_90
2,8,78,3621,0.840,0.109956,0.121,0.130,False,False
14,27,78,11486,0.247,0.109956,0.121,0.130,False,False
19,27,78,30457,0.397,0.109956,0.121,0.130,False,False
20,27,78,42730,0.298,0.109956,0.121,0.130,False,False
21,27,78,44099,0.403,0.109956,0.121,0.130,False,False
...,...,...,...,...,...,...,...,...,...
15388,21184,4412,154952,1.600,1.127250,1.276,1.445,False,False
15389,21661,4412,115606,1.600,1.127250,1.276,1.445,False,False
15627,21661,4527,146662,1.212,1.158333,1.202,1.260,False,True
15629,21812,4527,154363,1.212,1.158333,1.202,1.260,False,True


In [95]:
# 2) Список партнеров с их рекламными кампаниями, которые прошли точку невозврата с точки зрения окупаемости с точностью 90%
df_campaigns_fin_false90

,user_id,offer_id,campaign_id,payback_day,mean,up_bound75,up_bound90,passed_75,passed_90
2,8,78,3621,0.840,0.109956,0.121,0.130,False,False
14,27,78,11486,0.247,0.109956,0.121,0.130,False,False
19,27,78,30457,0.397,0.109956,0.121,0.130,False,False
20,27,78,42730,0.298,0.109956,0.121,0.130,False,False
21,27,78,44099,0.403,0.109956,0.121,0.130,False,False
...,...,...,...,...,...,...,...,...,...
15262,17754,3446,150478,0.867,0.804250,0.808,0.811,False,False
15263,17754,3446,153910,0.867,0.804250,0.808,0.811,False,False
15264,17754,3446,155824,0.865,0.804250,0.808,0.811,False,False
15388,21184,4412,154952,1.600,1.127250,1.276,1.445,False,False


In [97]:
# 2) Список партнеров с их рекламными кампаниями, которые успешно окупаются

df_campaigns_fin_true

,user_id,offer_id,campaign_id,payback_day,mean,up_bound75,up_bound90,passed_75,passed_90
0,1,78,65607,0.076,0.109956,0.121,0.13,True,True
1,8,78,2859,0.118,0.109956,0.121,0.13,True,True
3,8,78,7028,0.000,0.109956,0.121,0.13,True,True
4,8,78,7196,0.000,0.109956,0.121,0.13,True,True
5,8,78,14356,0.000,0.109956,0.121,0.13,True,True
...,...,...,...,...,...,...,...,...,...
15641,22984,4261,154770,0.800,0.800000,0.800,0.80,True,True
15642,25307,4261,152929,0.800,0.800000,0.800,0.80,True,True
15643,23379,3759,147465,0.800,0.800000,0.800,0.80,True,True
15644,23690,3759,145421,0.800,0.800000,0.800,0.80,True,True


In [99]:
# 3) Партнеры и их кампании, у которых подписки не окупаются никогда

no_payback_df = pd.read_csv(r'nopayback.csv')
no_payback_df

,user_id,offer_id,campaign_id,total_profit,total_expense,payback_day
0,1,3144,65608,0,1500,inf
1,43,3227,1089,0,8000,inf
2,43,3271,9088,0,3500,inf
3,43,6242,942,0,2500,inf
4,43,9339,81,0,2600,inf
5,43,9340,32712,0,5000,inf
6,43,9340,224086,0,2500,inf
7,44,868,99053,0,300,inf
8,44,4431,59924,0,3500,inf
9,50,3049,125564,0,2000,inf


In [100]:
# Список сервисов, для которых у нас нет достаточного количества данных для статистически достоверного прогноза среднего 
# срока окупаемости подписок партнера на сервисе
no_data_ids

[152,
 153,
 157,
 473,
 621,
 629,
 834,
 856,
 875,
 973,
 1104,
 1162,
 1233,
 1237,
 1272,
 1301,
 1342,
 1380,
 1600,
 1714,
 2045,
 2178,
 2229,
 2256,
 2277,
 2313,
 2346,
 2347,
 2467,
 2471,
 2472,
 2473,
 2482,
 2495,
 2496,
 2529,
 2532,
 2682,
 2685,
 2697,
 2830,
 2846,
 2892,
 2915,
 2933,
 2934,
 2951,
 2958,
 2963,
 2969,
 2970,
 2971,
 2978,
 2985,
 2992,
 2995,
 2997,
 2998,
 3088,
 3089,
 3112,
 3115,
 3121,
 3122,
 3144,
 3160,
 3194,
 3209,
 3210,
 3295,
 3296,
 3356,
 3378,
 3390,
 3447,
 3448,
 3463,
 3473,
 3542,
 3550,
 3751,
 3753,
 3811,
 3817,
 3857,
 3885,
 3886,
 3919,
 3925,
 3926,
 3946,
 3949,
 3966,
 3977,
 3980,
 3982,
 3983,
 3997,
 4001,
 4019,
 4022,
 4027,
 4035,
 4039,
 4051,
 4108,
 4116,
 4117,
 4124,
 4125,
 4158,
 4173,
 4175,
 4177,
 4195,
 4203,
 4248,
 4249,
 4263,
 4264,
 4303,
 4334,
 4336,
 4337,
 4350,
 4356,
 4357,
 4361,
 4376,
 4385,
 4387,
 4393,
 4397,
 4413,
 4433,
 4434,
 4436,
 4438,
 4442,
 4444,
 4452,
 4459,
 4462,
 4466,
 4